<a href="https://colab.research.google.com/github/MiguelAMM42/EG/blob/main/EG_22_23_(interno)_7_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Engenharia Gramatical

Ano Letivo 22/23

Autor Tiago Baptista

# Aula 2

## Sumário



1.   O que é o lark ?
2.   Principais características
  - construções automáticas
  - ferramentas de visualização
  - suporte de ebnf
4.   lexer em lark
5.   Introdução aos transformers

## Notas
[lark cheatsheet](https://lark-parser.readthedocs.io/en/latest/_static/lark_cheatsheet.pdf)

[Documentação](https://lark-parser.readthedocs.io)

[lark common](https://github.com/lark-parser/lark/blob/master/lark/grammars/common.lark)


### Demo introdução a lark e Tranformers

####Exercício 1
Escreva uma gramática independente de contexto para definir uma lista possivelmente vazia de números inteiros separados por vírgulas.

Deve importar o módulo **Lark** para gerar um parser que ao reconhecer uma frase gere a respetiva árvore de parsing.

Use a função de *pretty print* do módulo *Lark* para imprimir a árvore construida.

Nesta primeira versão não se pretende que faça nenhuma travessia à árvore, 
mas em contrapartida pretende-se que escreva várias versões da GIC para usar extended-BNF (uso de operadores para descrever repetições) e para usar apenas BNF (uso de recursividade par descrever repetições). 




In [6]:
!pip install lark
from lark import Lark,Token
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

grammar1 = '''
//Regras Sintaticas
start: PE ( | elementos) PD
elementos : NUMERO (VIR NUMERO)*
// elemento : NUMERO 
//Regras Lexicográficas
NUMERO:"0".."9"+ // [0-9]+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
grammar5 = '''
//Regras Sintaticas
start: PE ( |elemento (VIR elemento)*) PD
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar2 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elemento VIR elementos
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar3 = '''
//Regras Sintaticas
start: PE elementos PD
elementos: elementos VIR elemento
        |
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar4 = '''
//Regras Sintaticas
start: PE ( |elementos) PD
elementos: elemento VIR elementos
         | elemento
elemento : NUMERO 
//Regras Lexicograficas
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''

grammar = '''
//Regras Sintaticas
start: PE elemento* PD
elemento : NUMERO (VIR NUMERO)*
//Regras Lexicográficas
NUMERO:"0".."9"+ 
PE:"["
PD:"]"
VIR:","
//Tratamento dos espaços em branco
%import common.WS
%ignore WS
'''
# Soma da lista, encontrar o maior da lista
class ExemploTransformer(Transformer):
  def start(self,elementos):
    print("start")
    
  def elemento(self,elemento):
    print("elemento soma")
    print(sum(elemento))
    
  def NUMERO (self,numero):
    print("numero")
    print(numero)
    return int(numero)
    
  def PE(self,pe):
    print("pe")
    print(pe)
    return str(pe)

  def PD(self,pd):
    print("pd")
    print(pd)
    return str(pd)

  def VIR(self,vir):
    print("vir")
    print(vir)
    return Discard

  pass

frase = "[1,23,345]"

p = Lark(grammar)   #não muito bem
#p = Lark(grammar1)  #recomendada
#p = Lark(grammar5)  #boa
#p = Lark(grammar2)  #incorreta
#p = Lark(grammar3)  #incorreta
#p = Lark(grammar4)   #aceitável

tree = p.parse(frase)
print(tree.pretty())
#for element in tree.children:
  #print(element)


data = ExemploTransformer().transform(tree) # chamar o transformer para obter
#print(data)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
start
  [
  elemento
    1
    ,
    23
    ,
    345
  ]

pe
[
numero
1
vir
,
numero
23
vir
,
numero
345
elemento soma
369
pd
]
start


####Exercício 2
Retome uma das GIC escrita em extended-BNF para definir uma linguagem que permita escrever uma lista possívelmente vazia de números inteiros separados por vírgulas,
mas agora use o respetivo Transformer e escreva funções para visitar os nodos da árvore de parsing e escrever o conteúdo de cada uma.


In [5]:
!pip install lark
from lark import Lark
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

class MyTransformer(Transformer):
    def start(self, items):
        output={}
        #output["elementos"] = items[1]
        #soma = sum(items[1])
        #output["soma"] = soma
        print("raiz")
        return (output)

    def elemento(self, elemento):
        print(elemento)
        #r = list(filter(lambda x: x!=',', elemento))
        #return r

    def VIR(self,op):
        print(str(op))
        #return Discard

    def NUMERO(self,numero):
        print(str(numero))

    def PE(self,pe):
        print(str(pe))

    def PD(self,pd):
        print(str(pd))

## Primeiro precisamos da GIC
grammar = '''
start: PE elemento* PD
elemento : primeiro resto 
primeiro : NUMERO
resto : (VIR NUMERO)*
NUMERO:"0".."9"+
PE:"["
PD:"]"
VIR:","

%import common.WS
%ignore WS
'''

frase = "[1,2,3]"
p = Lark(grammar)

parse_tree = p.parse(frase)
print(parse_tree.pretty())

data = MyTransformer().transform(parse_tree)
print(data)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
start
  [
  elemento
    primeiro	1
    resto
      ,
      2
      ,
      3
  ]

[
1
,
2
,
3
[Tree(Token('RULE', 'primeiro'), [None]), Tree(Token('RULE', 'resto'), [None, None, None, None])]
]
raiz
{}


#### Exercício 3


Desenvolva uma GIC para definir uma linguagem que permita escrever listas
mistas de números e palavras, de tal forma que as 3 frases abaixo sejam frases
válidas dessa linguagem:

LISTA 1 .

Lista aaa .

Lista 1, 2, agora, 3, 4, fim, 7, 8.

Resolva as seguintes alíneas recorrendo ao uso de Transformer :

(a) Calcule, retorne e imprima quantos elementos existem numa lista.

(b) Identifique, retorne e imprima o elemento que mais se repete numa lista.

(c) Calcule, retorne e imprima a soma de todos os números que se encontrem entre as palavras
agora e fim de uma lista.

(d) Verifique se uma lista é válida de acordo com os seguintes parâmetros :

- Caso a palavra "agora" ocorra então a palavra "fim" também tem de aparecer posteriormente na lista e o contrário também.
- Entre "agora" e "fim" tem de aparecer pelo menos um número.


In [40]:
!pip install lark
from lark import Lark
from lark.tree import pydot__tree_to_png
from lark import Transformer
from lark import Discard

class MyTransformer(Transformer):

    def __init__(self):
       self.sum = 0 ## tmb pode ser inicializado no start


    def start(self, items):
        output={}
        #print(items[0])
        output["sum"] = self.sum
        #soma = sum(items[1])
        #output["soma"] = soma
        print("raiz")
        return (output)

    def elemento(self, elemento):
        print(elemento)
        return elemento


    def simplex(self, simple):
        print(simple)
        r = list(filter(lambda x: x!=',', simple))
        return simple

    def complex(self, comple):
        print(comple)
        r = list(filter(lambda x: x!=',', comple))
        return comple

    def SEP(self,sep):
        print(sep)
        return Discard

    def PONTO(self,pnt):
        print(pnt)
        return Discard


    def AGORA(self,ag):
        print(str(ag))
        return Discard

    def FIM(self,f):
        print(str(f))
        return f

    def VIR(self,op):
        print(str(op))
        return Discard

    def ELEM(self,elem):
        print(str(elem))
        print(type(elem))
        self.sum += elem
        return elem

    def LISTA(self,lL):
        print(str(lL))
        return Discard

## Primeiro precisamos da GIC
grammar = '''
start: LISTA elemento* 
elemento : simple | complex
simple : (ELEM VIR)* ELEM PONTO
complex : (ELEM VIR)+ AGORA VIR (ELEM VIR)+ FIM PONTO
ELEM:"0".."9"+ | "a".."z"+
AGORA: "agora"
FIM: "fim"
LISTA: "LISTA" | "Lista"
VIR:","
PONTO : "."

%import common.WS
%ignore WS
'''

## Fazer com vários agora e fim 

#frase = "LISTA 1"
frase = "Lista 1,2, agora, 3,4,fim,7,8."
p = Lark(grammar)

parse_tree = p.parse(frase)
print(parse_tree.pretty())

data = MyTransformer().transform(parse_tree)
print(data)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
start
  Lista
  elemento
    simple
      1
      ,
      2
      ,
      agora
      ,
      3
      ,
      4
      ,
      fim
      ,
      7
      ,
      8
      .

Lista
1
<class 'lark.lexer.Token'>


VisitError: ignored

# Aula 3

## Sumário

Exercício 1


